In [1]:
import json;import datetime;from datetime import timedelta;import os, zipfile;
import pandas as pd;from scipy.optimize import curve_fit;from scipy.stats import chisquare
import numpy as np;import matplotlib.pyplot as plt
from LifeBuddyWebApp import create_app, db
from LifeBuddyWebApp.models import User, Post, Health_description, Health_measure
proxyApp=create_app()
ctx=proxyApp.app_context()
ctx.push()
from sqlalchemy import func

from bokeh.plotting import figure, output_file, show
import pandas as pd
from bokeh.resources import CDN
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Text, Span
from bokeh.io import curdoc
from bokeh.themes import built_in_themes

from bokeh.layouts import row
import time
import pytz
import zoneinfo
from pytz import timezone

C:\tools\miniconda3\envs\lifebuddy\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [2]:
#filter on user data only
base_query_health_description=db.session.query(Health_description).filter(Health_description.user_id==1)
# if current_user.id==2:
df_health_description=pd.read_sql(str(base_query_health_description)[:-1]+str(1),db.session.bind)
# else:
#     df_health_description=pd.read_sql(str(base_query_health_description)[:-1]+str(current_user.id),db.session.bind)

In [8]:
#This stays exactly
colNames=[i[len('health_description_'):] for i in list(df_health_description.columns)]
col_names_dict={i:j for i,j in zip(list(df_health_description.columns),colNames)}
df_health_description.rename(columns=col_names_dict, inplace=True)

In [9]:
df_health_description.head(1)

,id,datetime_of_activity,var_activity,var_type,var_periodicity,var_timezone_utc_delta_in_mins,time_stamp_utc,user_id,source_filename,metric1_carido,metric2_session_duration,metric3,metric4,metric5,note
0,1,2021-06-09 12:08:34.635000,YOGA,heart rate,seconds,-240.0,2021-08-22 15:13:51.668660,1,training-session-2021-06-09-6138678344-024ee81...,21.733782,359.0,NaN,None,None,None


In [10]:
def plot_text_format(x):
    return ('%.1f' % x).rstrip('0').rstrip('.')

In [16]:
#This stays exactly
df1=df_health_description.loc[df_health_description.metric1_carido<100]
df1=df1.sort_values(by=['datetime_of_activity'])
obs_x1=[ datetime.datetime.strptime(date_string,'%Y-%m-%d %H:%M:%S.%f') for date_string in df1.datetime_of_activity]
obs_y1=df1.metric1_carido
obs_y1_formatted=[ plot_text_format(i) for i in y_obs]

date_end=datetime.datetime.strptime(df1.datetime_of_activity.to_list()[-1],'%Y-%m-%d %H:%M:%S.%f')
date_end=date_end+ timedelta(days=1)
date_start=date_end- timedelta(days=7)

In [ ]:
source = ColumnDataSource(dict(x=x_obs, y=y_obs, text=y_obs_formatted))
p1=figure(x_axis_label='Time',x_axis_type='datetime',width=880, height=400, toolbar_location=None,
          tools='xwheel_zoom,xpan',active_scroll='xwheel_zoom',x_range=(date_start, date_end))
p1.yaxis.major_label_text_color='black'

glyph = Text(text="text", text_color="#d6fbf7")

p1.add_glyph(source, glyph)

In [13]:
#get x and y's for activity
df2=df_health_description.loc[(df_health_description.var_type=='Activity') ]
#1 time (obs_x)
obs_x2=[ datetime.datetime.strptime(date_string,'%Y-%m-%d %H:%M:%S.%f') for date_string in df2.datetime_of_activity]
#3 activity (obs_y2)
obs_y2=df2.var_activity.to_list()

In [21]:
obs_y2

['activity-walk', 'coffee', 'woke up']

In [35]:
fig1=figure(toolbar_location=None,tools='xwheel_zoom,xpan',active_scroll='xwheel_zoom',
            x_range=(date_start,date_end),y_range=(-10,90),width=800, height=300)

#add cardio_metric1
source1 = ColumnDataSource(dict(x=obs_x1, y=obs_y1, text=obs_y1_formatted))
glyph1 = Text(text="text", text_color="#d6fbf7")
fig1.add_glyph(source1, glyph1)

#add activity data
source2 = ColumnDataSource(dict(x=obs_x2, y=[80], text=obs_y2))
glyph2 = Text(text="text", text_color="#9aa6a5")
#add line for activity data
line_start_time=time.mktime(obs_x2[0].timetuple())*1000
important_time = Span(location=line_start_time, dimension='height', line_color="#9aa6a5", line_dash='dashed', line_width=3)

# line_start_time2=time.mktime((obs_x2[0]+timedelta(days=1)).timetuple())*1000
# important_time2 = Span(location=line_start_time2, dimension='height', line_color='blue', line_dash='dashed', line_width=3)

fig1.add_layout(important_time)
# fig1.add_layout(important_time2)

fig1.add_glyph(source2, glyph2)

curdoc().theme = 'contrast'


show(fig1)

In [37]:
fig1=figure(toolbar_location=None,tools='xwheel_zoom,xpan',active_scroll='xwheel_zoom',
            x_range=(date_start,date_end),y_range=(-10,90),width=800, height=300)

#add cardio_metric1
source1 = ColumnDataSource(dict(x=obs_x1, y=obs_y1, text=obs_y1_formatted))
glyph1 = Text(text="text", text_color="#d6fbf7")
fig1.add_glyph(source1, glyph1)

for a,b in zip(obs_x2,obs_y2):
    #add activity data
    source2 = ColumnDataSource(dict(x=[a], y=[80], text=[b]))
    glyph2 = Text(text="text", text_color="#9aa6a5")
    #add line for activity data
#     line_start_time=time.mktime(a.timetuple())*1000
    line_start_time=time.mktime(a.timetuple())*1000
    important_time = Span(location=line_start_time, dimension='height', line_color="#9aa6a5", line_dash='dashed', line_width=3)
    fig1.add_glyph(source2, glyph2)
    fig1.add_layout(important_time)
#     line_var_dict[str(a)+b]=(source2,glyph2,important_time)

fig1.add_glyph(source2, glyph2)

curdoc().theme = 'contrast'


show(fig1)

a:: <class 'datetime.datetime'> 2021-08-22 21:08:00
b:: <class 'str'> activity-walk
a:: <class 'datetime.datetime'> 2021-08-23 05:30:00
b:: <class 'str'> coffee
a:: <class 'datetime.datetime'> 2021-08-24 08:18:00
b:: <class 'str'> woke up


In [33]:
time.mktime(obs_x2[0].timetuple())*1000

1629659280000.0

In [32]:
obs_x2[0].timetuple()

time.struct_time(tm_year=2021, tm_mon=8, tm_mday=22, tm_hour=21, tm_min=8, tm_sec=0, tm_wday=6, tm_yday=234, tm_isdst=-1)